In [1]:


import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import os
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from ipywidgets import fixed, Layout, Button, Box
import scipy.optimize as spopt 
import io
from kmc.tools import *
%matplotlib widget
#%matplotlib inline

#plt.style.use(['labplot', 'labplot_note'])
try:
    with open('pathfile.txt','r') as p:
        for line in p:
            path_bash = line
except:
    path_bash = ''
#files = [i for i in os.listdir('.') if '.txt' in i]

run_but = widgets.ToggleButton(
    value=False,
    description='Read File',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

#dropdown = widgets.Dropdown(options=files, values=files[0],description='File name',disabled=False)

dropdown = widgets.FileUpload(
    accept='.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)


def trpl_plot(dataframe,bin_num,tot_len,dump_but,legend):
    plt.figure(5)
    plt.clf()
    fluorescence = dataframe['Time'].to_numpy(float)/1000
    hist, bins = np.histogram(fluorescence, bins=np.logspace(0,np.log10(max(fluorescence)), bin_num),density=True) #bin_num)
    
    
    plt.xlabel("Time (ns)")          #Defines the x axis' label parameters
    plt.ylabel("Intensity (a.u.)")   #Defines the y axis' label parameters
    plt.ylim([-.05,1.05])            #Defines the y range
    
    plt.plot(bins[:-1],hist/max(hist), label=legend)
    avg_lf = np.mean(fluorescence)
    
    print(f'The average lifetime is: {avg_lf:.2f} ns')
    print(f'We are using {100*len(fluorescence)/tot_len:.2f} % of the total data')
    
    plt.legend()
    plt.show()
    
    if dump_but:
        hist_data = np.column_stack([bins[:-1], hist/max(hist)])
        np.savetxt(path_bash+'/trpl.txt'     , hist_data, fmt=['%f','%f'])
    
    

def trpl(data,tot_len,particle_existence,**kwargs):
    
    particles_filter = {}
    particles_filter["singlet"] = [kwargs["singl_mats"], kwargs["singl_energy"], 
    kwargs["singl_deaths"], kwargs["singl_status"]]
    
    particles_filter["triplet"] = [kwargs["tripl_mats"], kwargs["tripl_energy"], 
    kwargs["tripl_deaths"], kwargs["tripl_status"]]
    
    particles_filter["hole"]    = [kwargs["hole_mats"], kwargs["hole_energy"], 
    kwargs["hole_deaths"], kwargs["hole_status"]]
    
    particles_filter["electron"] = [kwargs["elect_mats"], kwargs["elect_energy"], 
    kwargs["elect_deaths"], kwargs["elect_status"]]
    
    data_filtred = []
    for particle in particles_filter:
        if particle_existence[particle][0] != 0:
            mats,energy,deaths,status = particles_filter[particle]
            data_filtred.append(filter_per_particle(data,particle,deaths,mats,energy,status))
    data_filtred =  pd.concat(data_filtred)

    try:
        bin_slider = widgets.IntSlider(
        value=100,
        max=300,
        min=2,
        step=1,
        description='Number of Bins:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
        )
        

        dump_but = widgets.ToggleButton(
            value=False,
            description='Export Data',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Description',
            icon='check'
        )

        #display(widgets.HTML(value = r'<p style="font-size:24px"><b>Plot Options</b></p>'))
        #display(widgets.Label('Plot Options', layout=Layout(width='6000px')))
        
        #gettin which processes were selected to the trpl calculation
        legend = sorted([kwargs[i][0] for i in kwargs.keys() if 'deaths' in i and kwargs[i][0] != 'Absent'])
        legend = ' '.join(legend).capitalize() #capitalize(' '.join(legend))
        
        widgets.interact(trpl_plot,dataframe=fixed(data_filtred),bin_num = bin_slider,tot_len=fixed(tot_len),dump_but=dump_but,legend=fixed(legend))
        #i = widgets.interactive(trpl_plot,dataframe=fixed(data_filtred),bin_num = bin_slider,param_fit1=param_1,param_fit2=param_2,tot_len=fixed(tot_len),dump_but=dump_but)
        #func_input   = {"dataframe":fixed(data_filtred), "bin_num":bin_slider, "param_fit1":param_1, "param_fit2":param_2,"tot_len":fixed(tot_len),"dump_but":dump_but}
        #func_calling = widgets.interactive_output(trpl_plot, func_input)
        #v = widgets.VBox([param_1,param_2])
        #h = widgets.HBox([bin_slider,dump_but])
        #h = widgets.HBox([v,h])
        #h = widgets.VBox([h,i])
        #display(h,func_calling)
        
    except:
        print('error')
        pass


def make_hist_plot(histogram_data):
    try:
        names, counts = zip(*histogram_data)
        print(names, counts)
        x = np.arange(len(names))
        plt.figure(3)
        plt.clf()
        plt.bar(x, height=counts)
        plt.xticks(x, names)  
        plt.show()
    except:
        pass
    #x = np.arange(3)
    #plt.bar(x, height=[1,2,3])
    #plt.xticks(x, ['a','b','c']);
def Histogram_func(data,name_deaths,**kwargs):
    
    singlet_boxes = [kwargs["singl_box1"], kwargs["singl_box2"], 
    kwargs["singl_box3"], kwargs["singl_box4"],
    kwargs["singl_box5"], kwargs["singl_box6"]]

    triplet_boxes = [kwargs["tripl_box1"], kwargs["tripl_box2"], 
    kwargs["tripl_box3"], kwargs["tripl_box4"],
    kwargs["tripl_box5"], kwargs["tripl_box6"]]

    hole_boxes = [kwargs["hole_box1"], kwargs["hole_box2"], 
    kwargs["hole_box3"], kwargs["hole_box4"],
    kwargs["hole_box5"], kwargs["hole_box6"]]
    
    electron_boxes = [kwargs["elect_box1"], kwargs["elect_box2"], 
    kwargs["elect_box3"], kwargs["elect_box4"],
    kwargs["elect_box5"], kwargs["elect_box6"]]
    
    dump_but = kwargs["dump_but"]
    
    par_boxes = {}
    par_boxes['singlet']  = singlet_boxes
    par_boxes['triplet']  = triplet_boxes
    par_boxes['electron'] = electron_boxes
    par_boxes['hole']     = singlet_boxes
    
    histogram_data = []
    
    #looping over the boxes cheched, getting the filtered data and plotting
    for part in name_deaths:
        list_death = name_deaths[part]
        for i in range(len(list_death)):
            death = list_death[i]
            it_is_on = par_boxes[part][i]
            print(death,it_is_on)
            if it_is_on:
                data_part = data[part]
                data_filter = filter_data_list(data_part,'CausaMortis',[death])
                n_filter = len(data_filter)
                histogram_data.append([part+'_'+death,n_filter])
    if histogram_data:
        make_hist_plot(histogram_data)
        if dump_but:
            hist_data = np.array(histogram_data,dtype='str')
            np.savetxt(path_bash+'/histogram.txt' , hist_data, delimiter=' ', fmt='%s')
#button to download the data from histogram
dump_but_hist = widgets.ToggleButton(
    value=False,
    description='Export Data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    )



def Histogram_widget(data,tot_len,particle_existence,**kwargs):
    
    #filtering using the common filtering interface
    particles_filter = {}
    particles_filter["singlet"] = [kwargs["singl_mats"], kwargs["singl_energy"], 
    kwargs["singl_deaths"], kwargs["singl_status"]]
    
    particles_filter["triplet"] = [kwargs["tripl_mats"], kwargs["tripl_energy"], 
    kwargs["tripl_deaths"], kwargs["tripl_status"]]
    
    particles_filter["hole"]    = [kwargs["hole_mats"], kwargs["hole_energy"], 
    kwargs["hole_deaths"], kwargs["hole_status"]]
    
    particles_filter["electron"] = [kwargs["elect_mats"], kwargs["elect_energy"], 
    kwargs["elect_deaths"], kwargs["elect_status"]]

    data_filtred = {}
    for particle in particles_filter:
        if particle_existence[particle][0] != 0:
            mats,energy,deaths,status = particles_filter[particle]
            data_filtred[particle] = filter_per_particle(data,particle,deaths,mats,energy,status)  
    
    #getting the death processes relevant for each particle
    death_list = {}
    death_list["singlet"]  = kwargs["singl_deaths"]
    death_list["triplet"]  = kwargs["tripl_deaths"]
    death_list["electron"] = kwargs["elect_deaths"]
    death_list["hole"]     = kwargs["hole_deaths"]
    
    name_deaths = {}
    box_singlet,deaths_singlet   = get_boxes_for_a_particle(6,death_list["singlet"])
    box_triplet,deaths_triplet   = get_boxes_for_a_particle(6,death_list["triplet"])
    box_hole,deaths_hole         = get_boxes_for_a_particle(6,death_list["hole"])
    box_electron,deaths_electron = get_boxes_for_a_particle(6,death_list["electron"])

    name_deaths['singlet']  = deaths_singlet
    name_deaths['triplet']  = deaths_triplet
    name_deaths['hole']     = deaths_hole
    name_deaths['electron'] = deaths_electron
    
    #initializating the histogram widget by giving the boxes as input
    w_hi = widgets.interactive(Histogram_func,data=fixed(data_filtred),name_deaths=fixed(name_deaths),
                           singl_box1=box_singlet[0],singl_box2=box_singlet[1],
                           singl_box3=box_singlet[2],singl_box4=box_singlet[3],
                           singl_box5=box_singlet[4],singl_box6=box_singlet[5],
                           tripl_box1=box_triplet[0],tripl_box2=box_triplet[1],
                           tripl_box3=box_triplet[2],tripl_box4=box_triplet[3],
                           tripl_box5=box_triplet[4],tripl_box6=box_triplet[5],
                           hole_box1=box_hole[0],hole_box2=box_hole[1],
                           hole_box3=box_hole[2],hole_box4=box_hole[3],
                           hole_box5=box_hole[4],hole_box6=box_hole[5],  
                           elect_box1=box_electron[0],elect_box2=box_electron[1],
                           elect_box3=box_electron[2],elect_box4=box_electron[3],
                           elect_box5=box_electron[4],elect_box6=box_electron[5],
                           dump_but = dump_but_hist)    

    #organize the boxes by particle type
    def organize_boxes(num_ele,widget_int):
        labels = ['Singlet','Triplet', 'Hole', 'Electron']
        n_boxes = int(len(widget_int.children)/num_ele)
        #print(n_boxes)
        v = widgets.VBox([widgets.Label('')])
        for k in range(n_boxes):
            rang = range(num_ele*k,(num_ele)*(k+1)-1)
            print(labels[k],rang)
            h = widgets.HBox([widget_int.children[i] for i in rang])
            v = widgets.VBox([v,widgets.Label(labels[k]),h])
        v = widgets.VBox([v,widget_int.children[-1]])
        return v
                         
    v = organize_boxes(6,w_hi)                
    #h = widgets.HBox([w_hi.children[i] for i in range(len(w_hi.children)-1)])
    #v = widgets.VBox([widgets.Label('Singlet: '),h,w_hi.children[-1]])
    display(v,dump_but_hist)
    #display(w_hi)

def pre_load(file_name,run_button):
    #print('the file '+ list(file_name.keys())[0] + 'was loaded!')
    if run_button:
        name = list(file_name.keys())[0]
        data = file_name[name]['content']
        data = io.StringIO(data.decode('utf-8'))
        data = pd.read_csv(data, delim_whitespace=True)
        data = data[data.Time != 'END'] #removes the end of round lines
        
        #data = pd.read_csv(file_name, delim_whitespace=True)
        #data = data[data.Time != 'END'] #removes the end of round lines
        tot_len = np.shape(data)[0]
        unique_energy = data['Energy'].unique()

        max_energy = np.amax(unique_energy)
        min_energy = np.amin(unique_energy)
        
        ########################################    
        #defining widgets after pre_load process
        #for each particle there is two sets of widgets: 
        # 1) the ones attached to the table : [energy , mats,   deaths, status] 
        # 2) the ones attached to the widget.interactive : [energy2, mats2, deaths2, status2] 
        # Comunication between them is mediated by the report_ie_change function
        types         = {'singlet':0,'triplet':0,'hole':0,'electron':0}
        obserb_dics = {}
        for part in types:
            try:
                data_part = filter_data_list(data,'Type',[part])
                unique_mats   = data_part['Location'].unique()
                unique_death  = data_part['CausaMortis'].unique()
                unique_status = data_part['Status'].unique()
                energy , mats,   deaths, status  = get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death,unique_status)
                energy2, mats2, deaths2, status2 = get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death,unique_status)

                obserb_dics[part] = ([ mats, energy ,  deaths, status],[mats2, energy2 ,  deaths2, status2])
                ui = make_boxes(deaths,status,mats,energy)
                types[part] = [1,ui]
            except:
                min_energy,max_energy,unique_mats,unique_death,unique_status = 0,1,[0,1],['Absent'],['Absent']
                energy , mats,   deaths, status  = get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death,unique_status)
                energy2, mats2, deaths2, status2 = get_widgets_per_particle(min_energy,max_energy,unique_mats,unique_death,unique_status)
                obserb_dics[part] = ([ mats, energy ,  deaths, status],[mats2, energy2 ,  deaths2, status2])
                types[part] = [0,make_boxes(deaths,status,mats,energy)]
        #xx = [types[part][0] for part in types]
        #print(xx)
        ###########################

        #initializing table
        output = widgets.Output()
        tab_contents = ['SINGLET', 'TRIPLET', 'HOLE', 'ELECTRON']
        children     = [types['singlet'][1], types['triplet'][1], types['hole'][1], types['electron'][1]] #[types[part][1] for part in types] #[w_singlet]#,w_triplet,w_hole,w_electron]
        tab          = widgets.Tab()
        tab.children = children
        #setting up the titles of the table
        for i in range(len(children)):
            tab.set_title(i,tab_contents[i])
        display(tab,output)
        
        
        singl_mats, singl_energy ,  singl_deaths, singl_status = obserb_dics['singlet'][1]
        tripl_mats, tripl_energy ,  tripl_deaths, tripl_status = obserb_dics['triplet'][1]
        hole_mats , hole_energy  ,  hole_deaths , hole_status  = obserb_dics['hole'][1]
        elect_mats, elect_energy ,  elect_deaths, elect_status = obserb_dics['electron'][1]
        
        #Calling Histogram INTERACTIVE INTERFACE
        w_hist = widgets.interactive(Histogram_widget,data=fixed(data),tot_len=fixed(tot_len),particle_existence=fixed(types),
                                singl_mats=singl_mats, singl_energy=singl_energy,
                                singl_deaths=singl_deaths, singl_status=singl_status,
                                tripl_mats=tripl_mats, tripl_energy=tripl_energy,
                                tripl_deaths=tripl_deaths, tripl_status=tripl_status,
                                hole_mats=hole_mats, hole_energy=hole_energy,
                                hole_deaths=hole_deaths, hole_status=hole_status,
                                elect_mats=elect_mats, elect_energy=elect_energy,
                                elect_deaths=elect_deaths, elect_status=elect_status)
        
        #Calling TRPL INTERACTIVE INTERFACE
        w = widgets.interactive(trpl,data=fixed(data),tot_len=fixed(tot_len),particle_existence=fixed(types),
                                singl_mats=singl_mats, singl_energy=singl_energy,
                                singl_deaths=singl_deaths, singl_status=singl_status,
                                tripl_mats=tripl_mats, tripl_energy=tripl_energy,
                                tripl_deaths=tripl_deaths, tripl_status=tripl_status,
                                hole_mats=hole_mats, hole_energy=hole_energy,
                                hole_deaths=hole_deaths, hole_status=hole_status,
                                elect_mats=elect_mats, elect_energy=elect_energy,
                                elect_deaths=elect_deaths, elect_status=elect_status)
        
        accordion = widgets.Accordion(children=[w,w_hist], selected_index=None)
        accordion.set_title(0, 'TIME RESOLVED PHOTOLUMINESCENCE')
        accordion.set_title(1, 'HISTOGRAM')
        accordion.set_title(2, 'HEATMAP')
        display(accordion)
        #print(len(accordion.children[0].children),accordion.children[0].children)
        
        #this function updates the atribute value upon changes in the widget. Must remain defined inside of pre_load
        def report_ie_change(change,w,accordion,attrib,indx):
            with output:
                accordion.children[0].children[indx].value = attrib.value

        ##############################    
        # Caution! For some reason, cant put that in a loop. It stops to function
        #for j in [0,1,2,3]:
        #    obserb_dics['singlet'][0][j].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['singlet'][0][j],j))    
        obserb_dics['singlet'][0][0].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['singlet'][0][0],0))
        obserb_dics['singlet'][0][1].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['singlet'][0][1],1))
        obserb_dics['singlet'][0][2].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['singlet'][0][2],2))
        obserb_dics['singlet'][0][3].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['singlet'][0][3],3))
        
        
        obserb_dics['triplet'][0][0].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['triplet'][0][0],4))
        obserb_dics['triplet'][0][1].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['triplet'][0][1],5))
        obserb_dics['triplet'][0][2].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['triplet'][0][2],6))
        obserb_dics['triplet'][0][3].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['triplet'][0][3],7))
        
        obserb_dics['hole'][0][0].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['hole'][0][0],8))
        obserb_dics['hole'][0][1].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['hole'][0][1],9))
        obserb_dics['hole'][0][2].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['hole'][0][2],10))
        obserb_dics['hole'][0][3].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['hole'][0][3],11))
        
        obserb_dics['electron'][0][0].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['electron'][0][0],12))
        obserb_dics['electron'][0][1].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['electron'][0][1],13))
        obserb_dics['electron'][0][2].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['electron'][0][2],14))
        obserb_dics['electron'][0][3].observe(lambda change: report_ie_change(change,w,accordion,obserb_dics['electron'][0][3],15))
        #########################
        
        #needed because by default widget.interactive pops up the widges. Tuning them invisible here..
        for indx in range(len(w.children)-1):
            w.children[indx].close()
        #w.children[0].close()
    else:
        pass   


In [ ]:
display(widgets.HTML(value = r'<p style="font-size:24px"><b>DASHBOARD KMC</b></p>'))
i = widgets.interactive(pre_load, file_name=dropdown,run_button=run_but);
h = widgets.HBox([i.children[0],i.children[1]])
v = widgets.VBox([h,i.children[2]])
display(v)